In [1]:
import librosa
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.metrics import categorical_accuracy
import time

Using TensorFlow backend.


In [2]:
commands = next(os.walk('./data'))[1]
commands

['down',
 'eight',
 'five',
 'four',
 'go',
 'left',
 'nine',
 'no',
 'off',
 'on',
 'one',
 'right',
 'seven',
 'six',
 'stop',
 'three',
 'two',
 'up',
 'yes',
 'zero']

In [3]:
data = pd.DataFrame(data=None, columns=['command','file'])
for command in commands:
    drty = './data/'+command
    files = next(os.walk(drty))[2]
    cmd = np.repeat(command,len(next(os.walk(drty))[2]))
    temp = pd.DataFrame({'command':cmd, 'file':files})
    data = data.append(temp)
    
data.tail()

,command,file
2371,zero,ffd2ba2f_nohash_1.wav
2372,zero,ffd2ba2f_nohash_2.wav
2373,zero,ffd2ba2f_nohash_3.wav
2374,zero,ffd2ba2f_nohash_4.wav
2375,zero,fffcabd1_nohash_0.wav


In [4]:
command_lookup = np.eye(len(commands))
y = np.array([command_lookup[commands.index(i)] for i in data.command])

In [5]:
X = data

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(40245, 2)
(7103, 2)
(40245, 20)
(7103, 20)


In [8]:
def data_train_generator():
    for i in range(len(X_train)):
        filepath = './data/'+X_train.iloc[i].command+'/'+X_train.iloc[i].file
        x, sr = librosa.load(filepath)
        stft = np.abs(librosa.stft(x))
        mfcc = np.mean(librosa.feature.mfcc(y=x, sr=sr, n_mfcc=40).T, axis=0)
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sr).T, axis=0)
        mel = np.mean(librosa.feature.melspectrogram(y=x, sr=sr).T,axis=0)
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sr).T, axis=0)
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(x), sr=sr).T, axis=0)
        ext_features = np.hstack([mfcc, chroma, mel, contrast, tonnetz])
        # X = np.empty((0,193))
        # X = np.vstack([features, ext_features])
        X = np.array(ext_features)
        Y = command_lookup[commands.index(X_train.iloc[i].command)]
        yield X.reshape(1,193), Y.reshape(1,20)

In [9]:
train_gen = data_train_generator()
print((next(train_gen)[0]).shape)

(1, 193)


In [ ]:
x_test_fit = []
y_test_fit = []
for i in range(len(X_test)):
    filepath = './data/'+X_test.iloc[i].command+'/'+X_test.iloc[i].file
    x, sr = librosa.load(filepath)
    stft = np.abs(librosa.stft(x))
    mfcc = np.mean(librosa.feature.mfcc(y=x, sr=sr, n_mfcc=40).T, axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sr).T, axis=0)
    mel = np.mean(librosa.feature.melspectrogram(y=x, sr=sr).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sr).T, axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(x), sr=sr).T, axis=0)
    ext_features = np.hstack([mfcc, chroma, mel, contrast, tonnetz])
    features = np.empty((0,193))
    features = np.vstack([features, ext_features])
    label = command_lookup[commands.index(X_test.iloc[i].command)]
    x_test_fit.append(features)
    y_test_fit.append(label)
    if (i+1)%1000==0:
        print(i+1,' files read')
    
x_test_fit = np.array(x_test_fit)
y_test_fit = np.array(y_test_fit)

C:\Users\abjilani\AppData\Local\Continuum\Anaconda3\envs\dlnd-tf-lab\lib\site-packages\librosa\core\pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


In [87]:
x_test_fit = x_test_fit.reshape((x_test_fit.shape[0], x_test_fit.shape[2]))
y_test_fit.shape

IndexError: tuple index out of range

In [168]:
nn = Sequential()
nn.add(Dense(units=300, input_dim=193, activation='relu'))
nn.add(Dense(units=300, activation='relu'))
nn.add(Dropout(0.8))
nn.add(Dense(units=len(commands), activation='softmax'))
nn.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=[categorical_accuracy])

In [169]:
start_time = time.time()
nn.fit_generator(generator=train_gen, steps_per_epoch=len(X_train)/32, epochs=10, verbose=1, validation_data=(x_test_fit, y_test_fit))
end_time = time.time()
print(end_time, start_time)

Epoch 1/10
125/628 [====>.........................] - ETA: 61s - loss: 14.8144 - categorical_accuracy: 0.0800     

C:\Users\abjilani\AppData\Local\Continuum\Anaconda3\envs\dlnd-tf-lab\lib\site-packages\librosa\core\pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


629/628 [==============================] - 94s - loss: 15.2440 - categorical_accuracy: 0.0541 - val_loss: 15.2671 - val_categorical_accuracy: 0.0528

KeyboardInterrupt: 

In [113]:
len(X_train)/32

1257.65625

In [149]:
print(end_time-start_time)

1786.1418769359589
